In [121]:
import numpy as np
from io import StringIO 
import matplotlib.pyplot as plt
from scipy.stats import norm

In [122]:
# Zadanie 5
import numpy as np
from numpy.core.fromnumeric import shape

def dmtest(e1, e2, h=1, lossf='AE'):
    e1 = np.array(e1)
    e2 = np.array(e2)
    T = len(e1)
    if lossf == 'AE':
        d = np.abs(e1) - np.abs(e2)
    else: # lossf == 'SE'
        d = e1**2 - e2**2
    dMean = np.mean(d)
    gamma0 = np.var(d)
    if h > 1:
        raise NotImplementedError()
    else:
        varD = gamma0

    DM = dMean / np.sqrt((1 / T) * varD)
    return DM



def forecast_arx(DATA,loc_index,save_beta,hour=0,week_day=0):
    # DATA: 8-column matrix (date, hour, price, load forecast, Sat, Sun, Mon dummy, p_min)
    # Select data to be used
    # print(DATA[-1, :])
    price = DATA[:-1, 2]             # For day d (d-1, ...)
    price_min = DATA[:-1, 11]         # For day d
    Dummies = DATA[1:, 4:11]          # Dummies for day d+1
    loadr = DATA[1:, 3]              # Load for day d+1

    # Take logarithms
    price = np.log(price)
    mc = np.mean(price)
    price -= mc                      # Remove mean(price)
    price_min = np.log(price_min)
    price_min -= np.mean(price_min)  # Remove mean(price)
    loadr = np.log(loadr)

    # Calibrate the ARX model
    y = price[7:]                    # For day d, d-1, ...
    # Define explanatory variables for calibration
    # without intercept
    X = np.vstack([price[6:-1], price[5:-2], price[:-7], price_min[6:-1],
                   loadr[6:-1], Dummies[6:-1, 0], Dummies[6:-1, 1], Dummies[6:-1, 2], Dummies[6:-1, 3], Dummies[6:-1, 4], Dummies[6:-1, 5], Dummies[6:-1, 6]]).T
    # with intercept
    # X = np.vstack([np.ones(len(y)), price[6:-1], price[5:-2], price[:-7], price_min[6:-1],
    #                loadr[6:-1], Dummies[6:-1, 0], Dummies[6:-1, 1], Dummies[6:-1, 2]]).T
    # Define explanatory variables for day d+1
    # without intercept
    X_fut = np.hstack([price[-1], price[-2], price[-7], price_min[-1],
                    loadr[-1], Dummies[-1, 0], Dummies[-1, 1], Dummies[-1, 2], Dummies[-1, 3], Dummies[-1, 4], Dummies[-1, 5], Dummies[-1, 6]])
    # with intercept
    # X_fut = np.hstack([[1], price[-1], price[-2], price[-7], price_min[-1],
    #                    loadr[-1], Dummies[-1, 0], Dummies[-1, 1], Dummies[-1, 2]])
    # if index > 360 pomijać beta = np.linalg.lstsq(X, y, rcond=None)[0]
    if(loc_index < 360):
        loc_index = loc_index + 1
        beta = np.linalg.lstsq(X, y, rcond=None)[0]  # Estimate the ARX model
        save_beta[hour,:] = beta
        prog = np.dot(beta, X_fut)                   # Compute a step-ahead forecast
        return [np.exp(prog + mc),loc_index,save_beta]                     # Convert to price level
    else:
        prog = np.dot(save_beta[hour,:], X_fut)                   # Compute a step-ahead forecast
        return [np.exp(prog + mc),loc_index,save_beta]                     # Convert to price level


def forecast_narx(DATA):
    import keras
    from keras.layers import Input, Dense
    from keras.models import Model
    from keras.callbacks import EarlyStopping
    # DATA: 8-column matrix (date, hour, price, load forecast, Sat, Sun, Mon dummy, p_min)
    # Select data to be used
    # print(DATA[-1, :])
    price = DATA[:-1, 2]             # For day d (d-1, ...)
    price_min = DATA[:-1, 11]         # For day d
    Dummies = DATA[1:, 4:10]          # Dummies for day d+1
    loadr = DATA[1:, 3]              # Load for day d+1

    # Take logarithms
    price = np.log(price)
    mc = np.mean(price)
    price -= mc                      # Remove mean(price)
    price_min = np.log(price_min)
    price_min -= np.mean(price_min)  # Remove mean(price)
    loadr = np.log(loadr)

    # Calibrate the ARX model
    y = price[7:]                    # For day d, d-1, ...
    # Define explanatory variables for calibration
    '''
    dodać pozostałe Dummies 0-6
    '''
    print(shape(Dummies))

    X = np.vstack([price[6:-1], price[5:-2], price[:-7], price_min[6:-1],
                   loadr[6:-1], Dummies[6:-1, 0], Dummies[6:-1, 1], Dummies[6:-1, 2], Dummies[6:-1, 3], Dummies[6:-1, 4], Dummies[6:-1, 5], Dummies[6:-1, 6]]).T
    # Define explanatory variables for day d+1
    X_fut = np.hstack([price[-1], price[-2], price[-7], price_min[-1],
                       loadr[-1], Dummies[-1, 0], Dummies[-1, 1], Dummies[-1, 2], Dummies[-1, 3], Dummies[-1, 4], Dummies[-1, 5], Dummies[-1, 6]])

    # Define Neural Network model
    inputs = Input(shape=(X.shape[1], ))                  # Input layer
    hidden = Dense(units=20, activation='sigmoid')(inputs)# Hidden layer (20 neurons)
    outputs = Dense(units=1, activation='linear')(hidden) # Output layer
    model = keras.Model(inputs=inputs, outputs=outputs)
    # callbacks = [EarlyStopping(patience=20, restore_best_weights=True)]
    callbacks = []
    model.compile(loss='MAE', optimizer='ADAM')           # Compile model
    model.fit(X, y, batch_size=64, epochs=500, verbose=0, # Fit to data
              validation_split=.0, shuffle=False, callbacks=callbacks)
    prog = model.predict(np.array(X_fut, ndmin=2))        # Compute a step-ahead forecast

    return np.exp(prog + mc)                     # Convert to price level

def forecast_naive(DATA):
    if np.sum(DATA[-1, 4:7]) > 0:
        return DATA[-8, 2]
    return DATA[-2, 2]

import numpy as np
from calendar import weekday
from time import time as t

def task(argtup):
    '''Helper function for multi-core NARX'''
    data, startd, endd, j, hour = argtup
    data_h = data[hour::24, :]
    ts = t()
    task_output = forecast_narx(data_h[startd + j:endd + j + 1, :])
    print(f'{j}\t{hour}\t{t() - ts}')
    return task_output 

def epf_arx(data, Ndays, startd, endd, forecast_type='naive',loc_index=0,save_beta=0):
    if forecast_type.lower() == 'narx':      # forecst_narx imports additional libraries, importing
        print(":)")
        # from forecast import forecast_narx   # here ensures that they are not needed for ARX or naive
    elif forecast_type.lower() == 'narx_mc': # multi-core variant
        from multiprocessing import Pool
    # DATA:   4-column matrix (date, hour, price, load forecast)
    # RESULT: 4-column matrix (date, hour, price, forecasted price)
    first_day = str(int(data[0, 0]))
    first_day = (int(e) for e in (first_day[:4], first_day[4:6], first_day[6:]))
    i = weekday(*first_day) # Weekday of starting day: 0 - Monday, ..., 6 - Sunday
    N = len(data) // 24
    data = np.hstack([data, np.zeros((N*24, 9))]) # Append 'data' matrix with daily dummies & p_min
    for j in range(N):

        if i % 7 == 5:
            data[24*j:24*(j+1), 4] = 1 # Saturday dummy in 5th (index 4) column
        elif i % 7 == 6:
            data[24*j:24*(j+1), 5] = 1 # Sunday dummy in 6th column
        elif i % 7 == 0:
            data[24*j:24*(j+1), 6] = 1 # Monday dummy in 7th column
        elif i % 7 == 1:
            data[24*j:24*(j+1), 7] = 1 # wtorek
        elif i % 7 == 2:
            data[24*j:24*(j+1), 8] = 1 # sroda
        elif i % 7 == 3:
            data[24*j:24*(j+1), 9] = 1 # czwartek
        elif i % 7 == 4:
            data[24*j:24*(j+1), 10] = 1 # piatek

            '''
        # Dummies dla pozostałych dni
        '''
        i += 1
        data[24*j:24*(j+1), 11] = np.min(data[24*j:24*(j+1), 2]) # p_min in 8th column
        data[24*j:24*(j+1), 12] = data[24*j:24*(j+1), 2] # p_min in 8th column
        '''
        8 kolumna cena z godziny 24
        '''
    result = np.zeros((Ndays * 24, 4)) # Initialize `result` matrix
    result[:, :3] = data[endd*24:(endd + Ndays) * 24, :3]
    if forecast_type.lower() == 'narx_mc': # multi-core invocation of NARX model
        argtups = [(data, startd, endd, j, h) for j in range(Ndays) for h in range(24)]
        with Pool() as pool:   # Pool(N) uses N simultaneous processes
            res = pool.map(task, argtups)
        result[:, 3] = res
        return result
    # n_week_day = weekday(*first_day)
    for j in range(Ndays):     # For all days ...
        for hour in range(24): # ... compute 1-day ahead forecasts for each hour
            data_h = data[hour::24, :]
            # Compute forecasts for the hour
            if forecast_type.lower() == 'narx':
                ts = t()
                result[j * 24 + hour, 3] = forecast_narx(data_h[startd + j:endd + j + 1, :])
                print(f'{j}\t{hour}\t{t() - ts}')
            elif forecast_type.lower() == 'arx':
                result[j * 24 + hour, 3] = forecast_arx(data_h[startd + j :endd +j + 1, :],loc_index,save_beta,hour)[0]
                '''
                # Beta powinna być macierza 24x12
                ''' 
                loc_index = forecast_arx(data_h[startd + j :endd +j + 1, :],loc_index,save_beta)[1]
                save_beta[hour,:] = forecast_arx(data_h[startd + j :endd +j + 1, :],loc_index,save_beta)[2][hour,:]

            elif forecast_type.lower() == 'naive':
                result[j * 24 + hour, 3] = forecast_naive(data_h[startd + j:endd + j + 1, :])
        # if(n_week_day == 6):
        #     n_week_day = 0
        # else:
        #     n_week_day +=1
    return result

In [123]:

# Zadanie 4

loaded_data = []

# with open("GEFCOM.txt", "r") as file:
with open("NPdata_2013-2016.txt", "r") as file:
    for line in file:
        strintIO_line = StringIO(line)
        npLine = np.loadtxt(strintIO_line)
        loaded_data.append(npLine)

loaded_data = np.array(loaded_data)

data_361_1082 = loaded_data[361*24:1082*24,2]

# Naive forecasting
data_361_1082_naive_pred = np.zeros((np.shape(data_361_1082)))
data_361_1082_naive_pred[0:24] = data_361_1082[0:24]

for i in range(24,17304):
    data_361_1082_naive_pred[i] = data_361_1082[i-24]

# HW

s = 4
alpha = 0.1
beta = 0.1
gamma = 0.1

L = np.zeros(17304)
T = L.copy()
S = L.copy()
fx = L.copy()
# Set initial values of L, T and S
L[s-1] = np.sum(data_361_1082[:s]) / s
T[s-1] = np.sum(data_361_1082[s:2*s] - data_361_1082[:s]) / (s ** 2)
S[:s] = data_361_1082[:s] - L[s-1]

# Iterate to compute L(t), T(t), S(t) and FX(t)
for t in range(s, len(data_361_1082)-1):
    L[t] = alpha * (data_361_1082[t] - S[t-s]) + (1 - alpha) * (L[t-1] + T[t-1])
    T[t] = beta * (L[t] - L[t-1]) + (1 - beta) * T[t-1]
    S[t] = gamma * (data_361_1082[t] - L[t]) + (1 - gamma) * S[t-s]
    fx[t+1] = L[t] + T[t] + S[t - s + 1]

def dmtest(e1, e2, h=1, lossf='AE'):
    e1 = np.array(e1)
    e2 = np.array(e2)
    T = len(e1)
    # AE: r=1
    if lossf == 'AE':
        d = np.abs(e1) - np.abs(e2)
    # AE: r=2
    else: # lossf == 'SE'
        d = e1**2 - e2**2
    dMean = np.mean(d)
    gamma0 = np.var(d)
    if h > 1:
        raise NotImplementedError()
    else:
        varD = gamma0

    DM = dMean / np.sqrt((1 / T) * varD)
    return DM



In [124]:
# Stałe okno (zad 5)

# data = np.loadtxt('GEFCOM.txt')
data = np.loadtxt('NPdata_2013-2016.txt')
startd = 0   # First day of the calibration window (startd from Matlab minus 1)
endd = 360   # First day to forecast (equal to endd from Matlab)
Ndays = 722  # user provided number of days to be predicted
             # (max is 722 for GEFCom with endd=360)

loc_index = 0
save_beta = np.zeros((24,12))

# naive, arx, narx or narx_mc
forecast_type = 'arx'

# Estimate and compute forecasts of the ARX model
res = epf_arx(data[:, :4], Ndays, startd, endd, forecast_type,loc_index,save_beta)

res_arx_stale_okno = res[:,3]

In [125]:
def dmtest(e1, e2, h=1, lossf='AE'):
    e1 = np.array(e1)
    e2 = np.array(e2)
    T = len(e1)
    if lossf == 'AE':
        d = np.abs(e1) - np.abs(e2)
    else: # lossf == 'SE'
        d = e1**2 - e2**2
    dMean = np.mean(d)
    gamma0 = np.var(d)
    if h > 1:
        raise NotImplementedError()
    else:
        varD = gamma0

    DM = dMean / np.sqrt((1 / T) * varD)
    return DM

def forecast_arx(DATA):
    # DATA: 8-column matrix (date, hour, price, load forecast, Sat, Sun, Mon dummy, p_min)
    # Select data to be used
    # print(DATA[-1, :])
    price = DATA[:-1, 2]             # For day d (d-1, ...)
    price_min = DATA[:-1, 11]         # For day d
    Dummies = DATA[1:, 4:11]          # Dummies for day d+1
    loadr = DATA[1:, 3]              # Load for day d+1

    # Take logarithms
    price = np.log(price)
    mc = np.mean(price)
    price -= mc                      # Remove mean(price)
    price_min = np.log(price_min)
    price_min -= np.mean(price_min)  # Remove mean(price)
    loadr = np.log(loadr)

    # Calibrate the ARX model
    y = price[7:]                    # For day d, d-1, ...
    # Define explanatory variables for calibration
    # without intercept
    X = np.vstack([price[6:-1], price[5:-2], price[:-7], price_min[6:-1],
                   loadr[6:-1], Dummies[6:-1, 0], Dummies[6:-1, 1], Dummies[6:-1, 2], Dummies[6:-1, 3], Dummies[6:-1, 4], Dummies[6:-1, 5], Dummies[6:-1, 6]]).T
    # with intercept
    # X = np.vstack([np.ones(len(y)), price[6:-1], price[5:-2], price[:-7], price_min[6:-1],
    #                loadr[6:-1], Dummies[6:-1, 0], Dummies[6:-1, 1], Dummies[6:-1, 2]]).T
    # Define explanatory variables for day d+1
    # without intercept
    X_fut = np.hstack([price[-1], price[-2], price[-7], price_min[-1],
                    loadr[-1], Dummies[-1, 0], Dummies[-1, 1], Dummies[-1, 2], Dummies[-1, 3], Dummies[-1, 4], Dummies[-1, 5], Dummies[-1, 6]])
    # with intercept
    # X_fut = np.hstack([[1], price[-1], price[-2], price[-7], price_min[-1],
    #                    loadr[-1], Dummies[-1, 0], Dummies[-1, 1], Dummies[-1, 2]])
    beta = np.linalg.lstsq(X, y, rcond=None)[0]  # Estimate the ARX model
    prog = np.dot(beta, X_fut)                   # Compute a step-ahead forecast
    return np.exp(prog + mc)                     # Convert to price level

def forecast_narx(DATA):
    import keras
    from keras.layers import Input, Dense
    from keras.models import Model
    from keras.callbacks import EarlyStopping
    # DATA: 8-column matrix (date, hour, price, load forecast, Sat, Sun, Mon dummy, p_min)
    # Select data to be used
    # print(DATA[-1, :])
    price = DATA[:-1, 2]             # For day d (d-1, ...)
    price_min = DATA[:-1, 7]         # For day d
    Dummies = DATA[1:, 4:7]          # Dummies for day d+1
    loadr = DATA[1:, 3]              # Load for day d+1

    # Take logarithms
    price = np.log(price)
    mc = np.mean(price)
    price -= mc                      # Remove mean(price)
    price_min = np.log(price_min)
    price_min -= np.mean(price_min)  # Remove mean(price)
    loadr = np.log(loadr)

    # Calibrate the ARX model
    y = price[7:]                    # For day d, d-1, ...
    # Define explanatory variables for calibration
    X = np.vstack([price[6:-1], price[5:-2], price[:-7], price_min[6:-1],
                   loadr[6:-1], Dummies[6:-1, 0], Dummies[6:-1, 1], Dummies[6:-1, 2]]).T
    # Define explanatory variables for day d+1
    X_fut = np.hstack([price[-1], price[-2], price[-7], price_min[-1],
                       loadr[-1], Dummies[-1, 0], Dummies[-1, 1], Dummies[-1, 2]])

    # Define Neural Network model
    inputs = Input(shape=(X.shape[1], ))                  # Input layer
    hidden = Dense(units=20, activation='sigmoid')(inputs)# Hidden layer (20 neurons)
    outputs = Dense(units=1, activation='linear')(hidden) # Output layer
    model = keras.Model(inputs=inputs, outputs=outputs)
    # callbacks = [EarlyStopping(patience=20, restore_best_weights=True)]
    callbacks = []
    model.compile(loss='MAE', optimizer='ADAM')           # Compile model
    model.fit(X, y, batch_size=64, epochs=500, verbose=0, # Fit to data
              validation_split=.0, shuffle=False, callbacks=callbacks)
    prog = model.predict(np.array(X_fut, ndmin=2))        # Compute a step-ahead forecast

    return np.exp(prog + mc)                     # Convert to price level

def forecast_naive(DATA):
    if np.sum(DATA[-1, 4:7]) > 0:
        return DATA[-8, 2]
    return DATA[-2, 2]

import numpy as np
from calendar import weekday
from time import time as t

def task(argtup):
    '''Helper function for multi-core NARX'''
    data, startd, endd, j, hour = argtup
    data_h = data[hour::24, :]
    ts = t()
    task_output = forecast_narx(data_h[startd + j:endd + j + 1, :])
    print(f'{j}\t{hour}\t{t() - ts}')
    return task_output 

def epf_arx(data, Ndays, startd, endd, forecast_type='naive'):
    if forecast_type.lower() == 'narx':      # forecst_narx imports additional libraries, importing
        print(":)")
        # from forecast import forecast_narx   # here ensures that they are not needed for ARX or naive
    elif forecast_type.lower() == 'narx_mc': # multi-core variant
        from multiprocessing import Pool
    # DATA:   4-column matrix (date, hour, price, load forecast)
    # RESULT: 4-column matrix (date, hour, price, forecasted price)
    first_day = str(int(data[0, 0]))
    first_day = (int(e) for e in (first_day[:4], first_day[4:6], first_day[6:]))
    i = weekday(*first_day) # Weekday of starting day: 0 - Monday, ..., 6 - Sunday
    N = len(data) // 24
    data = np.hstack([data, np.zeros((N*24, 9))]) # Append 'data' matrix with daily dummies & p_min
    for j in range(N):
        if i % 7 == 5:
            data[24*j:24*(j+1), 4] = 1 # Saturday dummy in 5th (index 4) column
        elif i % 7 == 6:
            data[24*j:24*(j+1), 5] = 1 # Sunday dummy in 6th column
        elif i % 7 == 0:
            data[24*j:24*(j+1), 6] = 1 # Monday dummy in 7th column
        elif i % 7 == 1:
            data[24*j:24*(j+1), 7] = 1 # wtorek
        elif i % 7 == 2:
            data[24*j:24*(j+1), 8] = 1 # sroda
        elif i % 7 == 3:
            data[24*j:24*(j+1), 9] = 1 # czwartek
        elif i % 7 == 4:
            data[24*j:24*(j+1), 10] = 1 # piatek

        '''
        # Dummies dla pozostałych dni
        '''
        i += 1
        data[24*j:24*(j+1), 11] = np.min(data[24*j:24*(j+1), 2]) # p_min in 8th column
    result = np.zeros((Ndays * 24, 4)) # Initialize `result` matrix
    result[:, :3] = data[endd*24:(endd + Ndays) * 24, :3]
    if forecast_type.lower() == 'narx_mc': # multi-core invocation of NARX model
        argtups = [(data, startd, endd, j, h) for j in range(Ndays) for h in range(24)]
        with Pool() as pool:   # Pool(N) uses N simultaneous processes
            res = pool.map(task, argtups)
        result[:, 3] = res
        return result
    for j in range(Ndays):     # For all days ...
        for hour in range(24): # ... compute 1-day ahead forecasts for each hour
            data_h = data[hour::24, :]
            # Compute forecasts for the hour
            if forecast_type.lower() == 'narx':
                ts = t()
                result[j * 24 + hour, 3] = forecast_narx(data_h[startd + j:endd + j + 1, :])
                print(f'{j}\t{hour}\t{t() - ts}')
            elif forecast_type.lower() == 'arx':
                result[j * 24 + hour, 3] = forecast_arx(data_h[startd:endd + j + 1, :])
            elif forecast_type.lower() == 'naive':
                result[j * 24 + hour, 3] = forecast_naive(data_h[startd + j:endd + j + 1, :])

    return result

In [126]:
# Zadanie 6 rozszerzane okno
# data = np.loadtxt('GEFCOM.txt')
data = np.loadtxt('NPdata_2013-2016.txt')
startd = 0   # First day of the calibration window (startd from Matlab minus 1)
endd = 360   # First day to forecast (equal to endd from Matlab)
Ndays = 722  # user provided number of days to be predicted
             # (max is 722 for GEFCom with endd=360)

# naive, arx, narx or narx_mc
forecast_type = 'arx'

# Estimate and compute forecasts of the ARX model
res = epf_arx(data[:, :4], Ndays, startd, endd, forecast_type)
res_arx_rozszerzane_okno = res[:,3]

In [127]:
# WH vs ARX ze rozszerzonym dniowym oknem
rv = norm()

# rozszerzane_okno vs HW
temp_res=res_arx_rozszerzane_okno[2*s+8::24]
rozszerzane_okno_hw_DM_r_1 = dmtest(res_arx_rozszerzane_okno[2*s:17288], fx[2*s:17288])
rozszerzane_okno_hw_DM_r_2 = dmtest(temp_res[:721], fx[2*s+8::24], lossf="R2")

rv = norm()

print("Metoda ARX rozszerzane okno vs WH dla 8 rano: " + str(stale_okno_hw_DM_r_2))
print("Metoda ARX rozszerzane okno vs WH dla 8 rano p value: " + str(1 - rv.cdf(stale_okno_hw_DM_r_2)))
print("Metoda ARX rozszerzane okno vs WH dla 24h: " + str(rozszerzane_okno_hw_DM_r_1))
print("Metoda ARX rozszerzane okno vs WH dla 24h p value: " + str(1 - rv.cdf(rozszerzane_okno_hw_DM_r_1)))

# HW vs rozszerzane_okno
temp_res = res_arx_rozszerzane_okno[2*s+8::24]
hw_rozszerzane_okno_DM_r_1 = dmtest(fx[2*s:2*s+17288], res_arx_rozszerzane_okno[2*s:17296])
hw_rozszerzane_okno_DM_r_2 = dmtest(fx[2*s+8::24],temp_res[:721],  lossf="R2")

print("Metoda HW vs ARX rozszerzane okno dla 8 rano: " + str(hw_rozszerzane_okno_DM_r_2))
print("Metoda HW vs ARX rozszerzane okno dla 8 rano p-value: " + str(1-rv.cdf(hw_rozszerzane_okno_DM_r_2)))
print("Metoda HW vs ARX rozszerzane okno dla 24h: " + str(hw_rozszerzane_okno_DM_r_1))
print("Metoda HW vs ARX rozszerzane okno dla 24h p-value: " + str(1-rv.cdf(hw_rozszerzane_okno_DM_r_1)))

Metoda ARX rozszerzane okno vs WH dla 8 rano: 6.95139992813106
Metoda ARX rozszerzane okno vs WH dla 8 rano p value: 1.8084422848119175e-12
Metoda ARX rozszerzane okno vs WH dla 24h: 6.021033301531873
Metoda ARX rozszerzane okno vs WH dla 24h p value: 8.665355100134775e-10
Metoda HW vs ARX rozszerzane okno dla 8 rano: -3.5253972015815864
Metoda HW vs ARX rozszerzane okno dla 8 rano p-value: 0.9997885758726905
Metoda HW vs ARX rozszerzane okno dla 24h: -6.192178701124291
Metoda HW vs ARX rozszerzane okno dla 24h p-value: 0.9999999997033089


In [128]:
# naive vs ARX ze rozszerzonym dniowym oknem
rv = norm()

# rozszerzane_okno vs naive
temp_res=res_arx_rozszerzane_okno[8::24]
rozszerzane_okno_naive_DM_r_1 = dmtest(res_arx_rozszerzane_okno[:17304],data_361_1082_naive_pred)
rozszerzane_okno_naive_DM_r_2 = dmtest(temp_res[:721], data_361_1082_naive_pred[8::24], lossf="R2")

rv = norm()

print("Metoda ARX rozszerzane okno vs naive dla 8 rano: " + str(rozszerzane_okno_naive_DM_r_2))
print("Metoda ARX rozszerzane okno vs naive dla 8 rano p value: " + str(1 - rv.cdf(rozszerzane_okno_naive_DM_r_2)))
print("Metoda ARX rozszerzane okno vs naive dla 24h: " + str(rozszerzane_okno_naive_DM_r_1))
print("Metoda ARX rozszerzane okno vs naive dla 24h p value: " + str(1 - rv.cdf(rozszerzane_okno_naive_DM_r_1)))

# naive vs rozszerzane_okno
# temp_res = res_arx_rozszerzane_okno[2*s+8::24]
naive_rozszerzane_okno_DM_r_1 = dmtest(data_361_1082_naive_pred, res_arx_rozszerzane_okno[:17304])
naive_rozszerzane_okno_DM_r_2 = dmtest(data_361_1082_naive_pred[8::24],temp_res[:721],  lossf="R2")

print("Metoda naive vs ARX rozszerzane okno dla 8 rano: " + str(naive_rozszerzane_okno_DM_r_2))
print("Metoda naive vs ARX rozszerzane okno dla 8 rano p-value: " + str(1-rv.cdf(naive_rozszerzane_okno_DM_r_2)))
print("Metoda naive vs ARX rozszerzane okno dla 24h: " + str(naive_rozszerzane_okno_DM_r_1))
print("Metoda naive vs ARX rozszerzane okno dla 24h p-value: " + str(1-rv.cdf(naive_rozszerzane_okno_DM_r_1)))

Metoda ARX rozszerzane okno vs naive dla 8 rano: 1.0374939283973685
Metoda ARX rozszerzane okno vs naive dla 8 rano p value: 0.1497528619437043
Metoda ARX rozszerzane okno vs naive dla 24h: 10.41841790670619
Metoda ARX rozszerzane okno vs naive dla 24h p value: 0.0
Metoda naive vs ARX rozszerzane okno dla 8 rano: -1.0374939283973685
Metoda naive vs ARX rozszerzane okno dla 8 rano p-value: 0.8502471380562957
Metoda naive vs ARX rozszerzane okno dla 24h: -10.41841790670619
Metoda naive vs ARX rozszerzane okno dla 24h p-value: 1.0


In [129]:
# stale okno vs ARX ze rozszerzonym dniowym oknem
rv = norm()

# rozszerzane_okno vs stale_okno
temp_res=res_arx_rozszerzane_okno[8::24]
rozszerzane_okno_stale_okno_DM_r_1 = dmtest(res_arx_rozszerzane_okno,res_arx_stale_okno)
rozszerzane_okno_stale_okno_DM_r_2 = dmtest(res_arx_rozszerzane_okno[8::24], res_arx_stale_okno[8::24], lossf="R2")

rv = norm()

print("Metoda ARX rozszerzane okno vs stale okno dla 8 rano: " + str(rozszerzane_okno_stale_okno_DM_r_2))
print("Metoda ARX rozszerzane okno vs stale okno dla 8 rano p value: " + str(1 - rv.cdf(rozszerzane_okno_stale_okno_DM_r_2)))
print("Metoda ARX rozszerzane okno vs stale okno dla 24h: " + str(rozszerzane_okno_stale_okno_DM_r_1))
print("Metoda ARX rozszerzane okno vs stale okno dla 24h p value: " + str(1 - rv.cdf(rozszerzane_okno_stale_okno_DM_r_1)))

# stale okno vs rozszerzane_okno
# temp_res = res_arx_rozszerzane_okno[2*s+8::24]
stale_okno_rozszerzane_okno_DM_r_1 = dmtest(res_arx_stale_okno, res_arx_rozszerzane_okno)
stale_okno_rozszerzane_okno_DM_r_2 = dmtest(res_arx_stale_okno[8::24],res_arx_rozszerzane_okno[8::24],  lossf="R2")

print("Metoda stale okno vs ARX rozszerzane okno dla 8 rano: " + str(stale_okno_rozszerzane_okno_DM_r_2))
print("Metoda stale okno vs ARX rozszerzane okno dla 8 rano p-value: " + str(1-rv.cdf(stale_okno_rozszerzane_okno_DM_r_2)))
print("Metoda stale okno vs ARX rozszerzane okno dla 24h: " + str(stale_okno_rozszerzane_okno_DM_r_1))
print("Metoda stale okno vs ARX rozszerzane okno dla 24h p-value: " + str(1-rv.cdf(stale_okno_rozszerzane_okno_DM_r_1)))

Metoda ARX rozszerzane okno vs stale okno dla 8 rano: 9.738766071344145
Metoda ARX rozszerzane okno vs stale okno dla 8 rano p value: 0.0
Metoda ARX rozszerzane okno vs stale okno dla 24h: 9.076333292830443
Metoda ARX rozszerzane okno vs stale okno dla 24h p value: 0.0
Metoda stale okno vs ARX rozszerzane okno dla 8 rano: -9.738766071344145
Metoda stale okno vs ARX rozszerzane okno dla 8 rano p-value: 1.0
Metoda stale okno vs ARX rozszerzane okno dla 24h: -9.076333292830443
Metoda stale okno vs ARX rozszerzane okno dla 24h p-value: 1.0


In [130]:
# WH vs ARX ze stalym 360 dniowym oknem

rv = norm()

# stale_okno vs HW
temp_res=res[2*s+8::24,3]
stale_okno_hw_DM_r_1 = dmtest(res[2*s:17288,3], fx[2*s:17288])
stale_okno_hw_DM_r_2 = dmtest(temp_res[:721], fx[2*s+8::24], lossf="R2")

rv = norm()

print("Metoda ARX stale okno vs WH dla 8 rano: " + str(stale_okno_hw_DM_r_2))
print("Metoda ARX stale okno vs WH dla 8 rano p value: " + str(1 - rv.cdf(stale_okno_hw_DM_r_2)))
print("Metoda ARX stale okno vs WH dla 24h: " + str(stale_okno_hw_DM_r_1))
print("Metoda ARX stale okno vs WH dla 24h p value: " + str(1 - rv.cdf(stale_okno_hw_DM_r_1)))

# HW vs stale_okno
temp_res = res[2*s+8::24,3]
hw_stale_okno_DM_r_1 = dmtest(fx[2*s:2*s+17288], res[2*s:17296,3])
hw_stale_okno_DM_r_2 = dmtest(fx[2*s+8::24],temp_res[:721],  lossf="R2")

print("Metoda HW vs ARX stale okno dla 8 rano: " + str(hw_stale_okno_DM_r_2))
print("Metoda HW vs ARX stale okno dla 8 rano p-value: " + str(1-rv.cdf(hw_stale_okno_DM_r_2)))
print("Metoda HW vs ARX stale okno dla 24h: " + str(hw_stale_okno_DM_r_1))
print("Metoda HW vs ARX stale okno dla 24h p-value: " + str(1-rv.cdf(hw_stale_okno_DM_r_1)))

Metoda ARX stale okno vs WH dla 8 rano: 3.5253972015815864
Metoda ARX stale okno vs WH dla 8 rano p value: 0.00021142412730945637
Metoda ARX stale okno vs WH dla 24h: 6.021033301531873
Metoda ARX stale okno vs WH dla 24h p value: 8.665355100134775e-10
Metoda HW vs ARX stale okno dla 8 rano: -3.5253972015815864
Metoda HW vs ARX stale okno dla 8 rano p-value: 0.9997885758726905
Metoda HW vs ARX stale okno dla 24h: -6.192178701124291
Metoda HW vs ARX stale okno dla 24h p-value: 0.9999999997033089


In [131]:
# Naive vs ARX ze stalym 360 dniowym oknem

rv = norm()

naive_stale_okno_r1 = dmtest(data_361_1082_naive_pred,res[:17304,3])
naive_stale_okno_r2 = dmtest(data_361_1082_naive_pred[8::24], dane_arx_8_rano[:721],lossf="R2")
print("Metoda naive vs ARX stale okno dla 8 rano: " + str(naive_stale_okno_r2))
print("Metoda naive vs ARX stale okno dla 8 rano p value: " + str(1 - rv.cdf(naive_stale_okno_r2)))
print("Metoda naive vs ARX stale okno dla 24h: " + str(naive_stale_okno_r1))
print("Metoda naive vs ARX stale okno dla 24h p value: " + str(1 - rv.cdf(naive_stale_okno_r1)))

stale_okno_naive_r1 = dmtest(res[:17304,3], data_361_1082_naive_pred)

stale_okno_naive_r2 = dmtest(dane_arx_8_rano[:721], data_361_1082_naive_pred[8:17304:24], lossf="R2")
print("Metoda HW vs naive dla 8 rano: " + str(stale_okno_naive_r2))
print("Metoda HW vs naive dla 8 rano p-value: " + str(1-rv.cdf(stale_okno_naive_r2)))
print("Metoda HW vs naive dla 24h: " + str(stale_okno_naive_r1))
print("Metoda HW vs naive dla 24h p-value: " + str(1-rv.cdf(stale_okno_naive_r1)))

Metoda naive vs ARX stale okno dla 8 rano: 0.4085817930969391
Metoda naive vs ARX stale okno dla 8 rano p value: 0.341423297143721
Metoda naive vs ARX stale okno dla 24h: -10.41841790670619
Metoda naive vs ARX stale okno dla 24h p value: 1.0
Metoda HW vs naive dla 8 rano: -0.4085817930969391
Metoda HW vs naive dla 8 rano p-value: 0.658576702856279
Metoda HW vs naive dla 24h: 10.41841790670619
Metoda HW vs naive dla 24h p-value: 0.0


In [132]:
# Naive vs HW

naive_hw_DM_r_1 = dmtest(data_361_1082_naive_pred[2*s:], fx[2*s:])
naive_hw_DM_r_2 = dmtest(data_361_1082_naive_pred[2*s+8::24], fx[2*s+8::24], lossf="R2")

rv = norm()

print("Metoda naive vs HW dla 8 rano: " + str(naive_hw_DM_r_2))
p_naive_hw_DM_r_2 = 1 - rv.cdf(naive_hw_DM_r_2)
print("Metoda naive vs HW dla 8 rano p value: " + str(p_naive_hw_DM_r_2))
print("Metoda naive vs HW dla 24h: " + str(naive_hw_DM_r_1))
p_naive_hw_DM_r_1 = 1 - rv.cdf(naive_hw_DM_r_1)
print("Metoda naive vs HW dla 24h p value: " + str(p_naive_hw_DM_r_1))

# HW vs Naive

hw_naive_DM_r_1 = dmtest(fx[2*s:], data_361_1082_naive_pred[2*s:])
hw_naive_DM_r_2 = dmtest(fx[2*s+8::24],data_361_1082_naive_pred[2*s+8::24],  lossf="R2")

print("Metoda HW vs naive dla 8 rano: " + str(hw_naive_DM_r_2))
print("Metoda HW vs naive dla 8 rano p-value: " + str(1-rv.cdf(hw_naive_DM_r_2)))
print("Metoda HW vs naive dla 24h: " + str(hw_naive_DM_r_1))
print("Metoda HW vs naive dla 24h p-value: " + str(1-rv.cdf(hw_naive_DM_r_1)))

Metoda naive vs HW dla 8 rano: 3.0751912907866354
Metoda naive vs HW dla 8 rano p value: 0.0010518379086487117
Metoda naive vs HW dla 24h: 0.2648701506748458
Metoda naive vs HW dla 24h p value: 0.39555474680520375
Metoda HW vs naive dla 8 rano: -3.0751912907866354
Metoda HW vs naive dla 8 rano p-value: 0.9989481620913513
Metoda HW vs naive dla 24h: -0.2648701506748458
Metoda HW vs naive dla 24h p-value: 0.6044452531947963


In [133]:
from IPython import get_ipython
get_ipython().magic('reset -sf')